<a href="https://colab.research.google.com/github/NotAtitYet/Crackles-And-Wheeze-Detection/blob/main/mosaic_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### importing libraries
import os
import librosa
import numpy as np
import pandas as pd

In [ ]:
!unzip /content/Testfile.zip

Archive:  /content/Testfile.zip
replace Test/139_1b1_Pr_sc_Litt3200.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/139_1b1_Pr_sc_Litt3200.txt  
replace Test/124_1b1_Al_sc_Litt3200.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/124_1b1_Al_sc_Litt3200.txt  
replace Test/156_2b3_Al_mc_AKGC417L.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/156_2b3_Al_mc_AKGC417L.txt  
replace Test/133_2p3_Al_mc_AKGC417L.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/133_2p3_Al_mc_AKGC417L.txt  
replace Test/139_1b1_Pr_sc_Litt3200.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/139_1b1_Pr_sc_Litt3200.wav  
replace Test/124_1b1_Al_sc_Litt3200.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/124_1b1_Al_sc_Litt3200.wav  
replace Test/118_1b1_Lr_sc_Litt3200.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/118_1b1_Lr_sc_Litt3200.wav  
replace Test/133_2p3_Al_mc_AKGC417L.wav? [y]es, [n]o, [A]ll, [N]one, [r]

In [ ]:
def extract(file_path):
    # Load audio file
    audio, sample_rate = librosa.load(file_path, duration=20)

    # Convert audio to spectrogram
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128, hop_length=512, n_fft=2048)

    # Convert spectrogram to log scale (dB)
    log_spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)

    return log_spectrogram.T

In [ ]:
## Getting all the Audio and Text file names
directory = '/content/Test'
audio_file_names=[]
text_file_names=[]
for file in os.listdir(directory):
  if file.endswith('.wav'):
    audio_file_names.append(os.path.join(directory,file))
    name=file.split('.')[0]
    text_file_names.append(os.path.join(directory,name + '.txt'))

In [ ]:
df=pd.read_csv(text_file_names[1], sep='\t', header=None, names=['start_time', 'end_time'])
df

In [ ]:
audio_features=[]

# audio_labels=[]
for index in range(len(audio_file_names)):

  ## here we are receiving the features from the audio files as an array of size 128 X 862
  spec = extract(audio_file_names[index])

  # ## Now we need to read each corresponding text files for the labels of crackles and wheezes
  df=pd.read_csv(text_file_names[index], sep='\t', header=None, names=['start_time', 'end_time',''])
  # temp_label=[0,0]
  for i, row in df.iterrows():

  #   ## The start and end time are float values so we convert them into proper int values so as to take proper segments from it
    start_idx = int(float(row['start_time']) / 0.04644)
    end_idx = int(float(row['end_time']) / 0.04644)
    # if row['crackle']==1:
    #   temp_label[0]=1
    # if row['wheeze']==1:
    #   temp_label[1]=1
    # audio_labels.append(temp_label)

    ## now that we have our labels we map the segment of the audio features array to its corresponding label
    ## we do this so that our model can understand which feature is a crackle and which is a wheeze
    audio_frame = spec[:, start_idx:end_idx]
    audio_features.append(spec)


In [ ]:
# Convert the list of features and labels to numpy arrays
max_len = 390  # 20s / (512/22050) ≈ 390
audio_features = np.zeros((len(audio_features), 128, max_len))
for i in range(len(audio_features)):
    audio_features[i] = librosa.util.fix_length(audio_features[i], size=max_len, axis=1)
audio_features = np.expand_dims(audio_features, axis=-1)
# audio_labels = np.array(audio_labels)

In [ ]:
import tensorflow as tf

# Convert the numpy array to a TensorFlow tensor
features_tensor = tf.convert_to_tensor(audio_features)

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/MosaicPS1/model.h5')

In [ ]:
y_pred = new_model.predict(audio_features)

1/1 [==============================] - 7s 7s/step


In [ ]:
y_pred

array([[0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ],
       [0.51615775, 0.4785738 ]], dtype=float32)